In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import scipy
from scipy.optimize import minimize
from scipy import io

In [ ]:
# Load data
data = scipy.io.loadmat('trajectories.mat')
# Path
iid = data['iidind'][2:,]
rdw = data['rdwind'][2:,]
posr = np.zeros((iid.shape[0] - 1, iid.shape[1]))
posw = np.zeros((rdw.shape[0]-1,rdw.shape[1]))
n_path = posr.shape[1]
invis = []
label_iid =np.zeros((1, n_path)) # ideal responses
label_rdw = np.zeros((1, n_path)) # ideal responses
for tidx in range(n_path):
    # IID
    invisR = int(iid[0, tidx])
    invis.append(invisR)
    r = iid[1:, tidx].copy()
    r[-invisR + 1:] = np.nan
    posr[:, tidx] = r  
    label_iid[0, tidx] = 1 if np.nanmean(posr[:, tidx]) < 0 else 0
    # rdw 
    w = rdw[1:, tidx].copy()
    w[-invisR+1:] = np.nan
    posw[:,tidx] = w
    label_rdw[0, tidx] = 1 if posw[-invis[tidx],tidx]< 0 else 0

In [ ]:
label_iid_same = np.hstack((label_iid[:,0:50], label_iid[:,150:]))
label_iid_opposite = label_iid[:,50:150]
label_rdw_same = np.hstack((label_rdw[:,0:50], label_rdw[:,150:]))
label_rdw_opposite = label_rdw[:,50:150]
label_iid_same.shape, label_iid_opposite.shape, label_rdw_same.shape, label_rdw_opposite.shape

In [ ]:
posr_same = np.hstack((posr[:,0:50], posr[:,150:]))
posr_opposite = posr[:,50:150]
posw_same = np.hstack((posw[:,0:50], posw[:,150:]))
posw_opposite = posw[:,50:150]

posr_same.shape, posr_opposite.shape, posw_same.shape, posw_opposite.shape

### Load subjects responce

In [ ]:
resp_rdw = pd.read_csv('C:/Users/USER/Desktop/Paris Saclay/These/5_MODEL/BM_exp_1/path_check/resp/Subjects_28/resp_rdw.csv', header=None).to_numpy()
resp_iid = pd.read_csv('C:/Users/USER/Desktop/Paris Saclay/These/5_MODEL/BM_exp_1/path_check/resp/Subjects_28/resp_iid.csv', header=None).to_numpy()
resp_iid

In [ ]:
resp_iid_same =  np.vstack((resp_iid[0:50:,], resp_iid[150::,]))
resp_iid_opposite = resp_iid[50:150:,]
resp_rdw_same= np.vstack((resp_rdw[0:50:,], resp_rdw[150::,]))
resp_rdw_opposite = resp_rdw[50:150:,]

In [ ]:
# resp_iid_same_1 =  pd.read_csv('resp_iid_same.csv', header=None).to_numpy()
# resp_iid_diff_1 =pd.read_csv('resp_iid_diff.csv', header=None).to_numpy()
# resp_rdw_same_1= pd.read_csv('resp_rdw_same.csv', header=None).to_numpy()
# resp_rdw_dif_1 = pd.read_csv('resp_rdw_diff.csv', header=None).to_numpy()

In [ ]:
#plot
def plotting (posr,posw):
    n_path = posr.shape[1]
    for i in range(n_path):
    
        fig = plt.figure(figsize=(10, 5))
        inv = np.count_nonzero(np.isnan(posr[:, i]))+1
        if np.nanmean(posr[:, i]) < 0:
            mean_side = 'L'
        else : 
            mean_side = 'R'
        if posr[:, i][-inv] < 0 :
            lv = 'L'
        else :
            lv = 'R'
        if mean_side == 'L' and lv == "L" or mean_side == 'R' and lv == "R" :
            cond ="same"
        else: 
            cond = "opposite"   
        plt.subplot(2, 1, 1)
        plt.plot(range(posr[:, i].shape[0]), posr[:, i], label='iid', linewidth=1,alpha = 0.001)
        plt.plot(range(posw[:, i].shape[0]), posw[:, i], label='rdw', linewidth=1)
        plt.plot([len(posr[:, i]) - inv, len(posr[:, i]) - inv], plt.ylim(), '--')
        plt.hlines(0, 0, len(posr[:, i]),'k',  alpha=0.2)

        plt.subplots_adjust(hspace=0)
    #     plt.title(f"IID path ({mean_side}{lv}{cond})")
        plt.ylim(-15, 15)
        plt.xlim(0, 150)
        plt.ylabel("X position", fontsize=12)
        plt.legend()

        plt.subplots_adjust(hspace=0)
        plt.title(f"path {i} {cond}, (mu: {round(np.nanmean(posr[:, i]),3)}, sd: {round(np.nanstd(posr[:, i]),3)} ) ")
        plt.ylim(-15, 15)
        plt.xlim(0, 150)
        plt.ylabel("X position", fontsize=12)

In [ ]:
plotting (posr,posw)

In [ ]:
plotting (posr_opposite,posw_opposite)

In [ ]:
#plot
def plotting (posr,):
    n_path = posr.shape[1]
    for i in range(n_path):
    
        fig = plt.figure(figsize=(10, 5))
        inv = np.count_nonzero(np.isnan(posr[:, i]))+1
        if np.nanmean(posr[:, i]) < 0:
            mean_side = 'L'
        else : 
            mean_side = 'R'
        if posr[:, i][-inv] < 0 :
            lv = 'L'
        else :
            lv = 'R'
        if mean_side == 'L' and lv == "L" or mean_side == 'R' and lv == "R" :
            cond ="same"
        else: 
            cond = "opposite"   
        plt.subplot(2, 1, 1)
        plt.plot(range(posr[:, i].shape[0]), posr[:, i], label='iid', linewidth=1)
        plt.plot(posr[:, i].shape[0]-inv, posr[-inv, i], 'o-' ,color='k')
        
#         plt..plot(iid_ind[-f_m,i], iid_ind.shape[0]-(f_m+1), 'o-' ,color='k',label=f'last visibl {round(iid_ind[-f_m,i],3)}')
        
        plt.plot([len(posr[:, i]) - inv, len(posr[:, i]) - inv], plt.ylim(), '--')
        
        plt.hlines(0, 0, len(posr[:, i]),'k',  alpha=0.2)

        plt.subplots_adjust(hspace=0)
    #     plt.title(f"IID path ({mean_side}{lv}{cond})")
        plt.ylim(-15, 15)
        plt.xlim(0, 150)
        plt.ylabel("X position", fontsize=12)
        plt.legend()

        plt.subplots_adjust(hspace=0)
        plt.title(f"path {i} {cond}, (mu: {round(np.nanmean(posr[:, i]),3)}, sd: {round(np.nanstd(posr[:, i]),3)} ) ")
        plt.ylim(-15, 15)
        plt.xlim(0, 150)
        plt.ylabel("X position", fontsize=12)

In [ ]:
plotting (posr_opposite)

In [ ]:
resp_iid_opposite_mean = np.mean(resp_iid_opposite, axis=1)
resp_iid_opposite_mean

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plotting(posr, resp_iid_opposite):
    n_path = posr.shape[1]
    
    for i in range(n_path):     
        fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4), gridspec_kw={'height_ratios': [1, 2]})

        inv = np.count_nonzero(np.isnan(posr[:, i])) + 1
        mean_side = 'L' if np.nanmean(posr[:, i]) < 0 else 'R'
        lv = 'L' if posr[:, i][-inv] < 0 else 'R'
        cond = "same" if (mean_side == 'L' and lv == "L") or (mean_side == 'R' and lv == "R") else "opposite"
        
        # Count the number of 0s and 1s for each trajectory
        counts_1 = np.sum(resp_iid_opposite[i], axis=0)
        counts_0 = resp_iid_opposite.shape[1] - counts_1
        mean_last = round(np.nanmean(posr_opposite[-inv-4:, i]),3)

        # Plot Binary Responses (Top subplot - Bar chart)
        participant_indices = np.array([0, 1])  # Updated indices for columns 0 and 1
        colors = ['blue', 'green']
        width = 0.4  # Adjust the width of the bars
        bars_0 = axes[0].bar(participant_indices[0], counts_0, color=colors[0], label='Count 0')
        bars_1 = axes[0].bar(participant_indices[1], counts_1, color=colors[1], label='Count 1')

        axes[0].set_xlim(-0.5, 1.5)
        axes[0].set_ylim(0, 30)
        axes[0].set_xticks(participant_indices)
        axes[0].set_xticklabels(['right resp', 'left resp'])
        axes[0].set_xlabel("Number", fontsize=12)
        axes[0].set_ylabel("Respons", fontsize=12)
        axes[0].legend()
   

        # Annotate bar chart with values
        for bar, count in zip(bars_0 + bars_1, [counts_0, counts_1]):
            height = bar.get_height()
            axes[0].annotate(f'{count}', xy=(bar.get_x() + bar.get_width() / 2, height),
                             xytext=(0, 3), textcoords='offset points',
                             ha='center', va='bottom')

        # Plot Trajectory (Bottom subplot)
        axes[1].plot(range(posr[:, i].shape[0]), posr[:, i], label=f'iid (mu_last5 {mean_last})', linewidth=1)
        axes[1].plot(posr[:, i].shape[0]-inv, posr[-inv, i], 'o-' ,color='k')
        axes[1].hlines( 0, 0, len(posr[:, i]), 'k', alpha=0.2 )
        axes[1].set_ylim(-15, 15)
        axes[1].set_xlim(0, 150)  # Set y-limits for the trajectory plot
        axes[1].set_ylabel("X position", fontsize=12)
        axes[1].set_xlabel("Frame", fontsize=12)
        axes[1].legend()
        axes[0].set_title(f"path {i} {cond} ({mean_side}{lv}), (mu: {round(np.nanmean(posr[:, i]),3)}, sd: {round(np.nanstd(posr[:, i]),3)}, lv {round(posr[-inv, i],2)}) ")

        plt.tight_layout()
        plt.show()

# Example usage:
# plotting(posr_opposite, resp_iid_opposite)


In [ ]:
plotting(posr_opposite, resp_iid_opposite)

In [ ]:
plotting(posr_opposite, resp_iid_opposite)

In [ ]:
n_path = posr_opposite.shape[1]
trajectory_data_list = []

for i in range(n_path):  
    counts_1 = np.sum(resp_iid_opposite[i], axis=0)
    counts_0 = resp_iid_opposite.shape[1] - counts_1
    inv = np.count_nonzero(np.isnan(posr_opposite[:, i]))
    trajectory_data = {
        "inex path": i, 
        "path": 'iid',
        "cond": 'opposite', 
        "mean_cond": "left" if np.nanmean(posr_same[:, i])<0 else 'right',
        "inv" : inv,
        "Mean": round(np.mean(posr_opposite[0:-inv,i]),3),
        "Mean": round(np.nanmean(posr_opposite[:, i]), 3),
        "Mean_last15" : round(np.nanmean(posr_opposite[-inv-15:, i]),3),
        "Mean_last10" : round(np.nanmean(posr_opposite[-inv-10:, i]),3),
        "Mean_last5" : round(np.nanmean(posr_opposite[-inv-5:, i]),3),
        "Standard Deviation": round(np.nanstd(posr_opposite[:, i]), 3),
        'last visibl' : round(posr_opposite[-(inv+1), i],3),
        "Count 0 (right resp)": counts_0,
        "Count 1 (left resp)": counts_1,
        "correct" : counts_1 if round(np.mean(posr_opposite[0:-inv, i]), 3) < 0 else counts_0,
        "error": counts_1 if round(np.mean(posr_opposite[0:-inv, i]), 3) > 0 else counts_0,
        "mean=last15": "y" if np.sign(np.nanmean(posr_opposite[-inv-15:, i])) == np.sign(np.mean(posr_opposite[0:-inv,i])) else "n",
        "mean=last10": "y" if np.sign(np.nanmean(posr_opposite[-inv-10:, i])) == np.sign(np.mean(posr_opposite[0:-inv,i])) else "n",
        "mean=last5": "y" if np.sign(np.nanmean(posr_opposite[-inv-5:, i])) == np.sign(np.mean(posr_opposite[0:-inv,i])) else "n",
    }
    trajectory_data_list.append(trajectory_data)

# Create a Pandas DataFrame for all trajectories
df = pd.DataFrame(trajectory_data_list)
df

In [ ]:
df.to_csv('trajectory_info_opposite_28s.csv', index=False)

### IID same check

In [ ]:
resp_iid_same

In [ ]:
resp_iid_same_mean = np.mean(resp_iid_same, axis=1)
np.round(resp_iid_same_mean,2)

In [ ]:
resp_iid_same.shape

In [ ]:
plotting(posr_same, resp_iid_same)

In [ ]:
n_path = posr_same.shape[1]
trajectory_data_list = []

for i in range(n_path):  
    counts_1 = np.sum(resp_iid_same[i], axis=0)
    counts_0 = resp_iid_same.shape[1] - counts_1
    inv = np.count_nonzero(np.isnan(posr_same[:, i]))
    trajectory_data = {
        "inex path": i,
        "path": 'iid',
        "cond": 'same',
        "inv" : inv,
        "mean_cond": "left" if np.nanmean(posr_same[:, i])<0 else 'right',
        "Mean": round(np.nanmean(posr_same[:, i]), 3),
        "Mean_last15" : round(np.nanmean(posr_same[-inv-15:, i]),3),
        "Mean_last10" : round(np.nanmean(posr_same[-inv-10:, i]),3),
        "Mean_last5" : round(np.nanmean(posr_same[-inv-5:, i]),3),
        "Standard Deviation": round(np.nanstd(posr_same[:, i]), 3),
        'last visibl' : round(posr_same[-(inv+1), i],3),
        "Count 0 (right resp)": counts_0,
        "Count 1 (left resp)": counts_1,
        "correct" : counts_1 if round(np.mean(posr_same[0:-inv, i]), 3) < 0 else counts_0,
        "error": counts_1 if round(np.mean(posr_same[0:-inv, i]), 3) > 0 else counts_0,
        "mean=last15": "y" if np.sign(np.nanmean(posr_same[-inv-15:, i])) == np.sign(np.mean(posr_same[0:-inv,i])) else "n",
        "mean=last10": "y" if np.sign(np.nanmean(posr_same[-inv-10:, i])) == np.sign(np.mean(posr_same[0:-inv,i])) else "n",
        "mean=last5": "y" if np.sign(np.nanmean(posr_same[-inv-5:, i])) == np.sign(np.mean(posr_same[0:-inv,i])) else "n",
    }
    trajectory_data_list.append(trajectory_data)
# Create a Pandas DataFrame for all trajectories
df_same = pd.DataFrame(trajectory_data_list)
df_same

In [ ]:
df_same['inv'].tolist()

In [ ]:
df_same.to_csv('trajectory_info_same_28s.csv', index=False)